In [4]:
import gdsfactory as gf
from blocks import *
from comb_drive_tuning import *



In [5]:
xs_metal_wire = metal_wire(core_width=7, wire_width=6.3, mask_offset=5.0)
xs_metal_wire_wide = metal_wire(core_width=70, wire_width=68, mask_offset=5.0)
xs = cross_section_with_sleeves(0.43,5,radius=5)
xs_routing = cross_section_with_sleeves(0.43,5,radius=20)

In [6]:
def doubly_clamped_beam_with_ring_resonator(beam_length=400, beam_width=0.1,spring_length=20, spring_width=0.15,finger_num = 140):
    x_span_resonator = 100
    beam_spec = partial(doubly_clamped_beam_with_round_support,length=beam_length, width=beam_width,support_length=0.2,create_mask=True,mask_offset=5)
    spring_spec = partial(spring_pair_anchor_outside,spring_length=spring_length,spring_width=spring_width,mask_offset=5,spring_separation=4)
    spring_beam = doubly_clamped_beam_with_spring(beam_spec,spring_spec)
    spring_beam = tmp_merge_deep_etch_mask(spring_beam)
    c = gf.Component()
    # bridge_ref = c << bridge(mxn=(5,5),mask_offset=5)
    spring_beam_ref = c << spring_beam
    # bridge_ref.connect("E1", spring_beam_ref.ports["w1"],allow_width_mismatch=True)
    mfs_ref = c << movable_finger_support(length=finger_num*(0.2+0.3),mask_offset=5)
    mfs_ref.connect("E1", spring_beam_ref.ports["w1"],allow_width_mismatch=True)
    comb = combdrive_fingers(fingers=finger_num,finger_length=3.2,finger_gap=0.3,thickness=0.2,base_thickness=0.2,a_c=3,mask_offset=5)
    comb_ref = c << comb
    comb_ref.connect("e1", mfs_ref.ports["W1"],allow_width_mismatch=True)
    fhs = finger_hard_support(size=(10,finger_num*(0.2+0.3)),mask_offset=5, metal_offset=1, metal_layer='MTOP')
    fhs_ref = c << fhs
    fhs_ref.connect("E1", comb_ref.ports["w1"],allow_width_mismatch=True)
    # fhs_ref2 = c << fhs
    # fhs_ref2.connect("E1", spring_beam_ref.ports["e1"],allow_width_mismatch=True)
    pad_ = pad(size=(400,400), metal_offset=10,pad_layer='PADDING')
    pad_ref = c << pad_
    pad_ref.connect("E1", fhs_ref.ports["W1"],allow_width_mismatch=True, allow_type_mismatch=True)
    pad_ref.movex(-100)
    gf.routing.route_single(c, pad_ref.ports['E1'], fhs_ref.ports['W1'], cross_section=xs_metal_wire_wide, allow_width_mismatch=True, auto_taper=False)

    bfs_ref = c << beam_fixed_support(size=(100,100), metal_offset=1)
    bfs_ref.connect("E1", spring_beam_ref.ports["e1"],allow_width_mismatch=True)
    # pad_ref2 = c << pad_
    # pad_ref2.connect("E1", bfs_ref.ports["W1"],allow_width_mismatch=True)
    ring_beam_spacer = vertical_spacer(length=0.1)
    ring_ref = c << ring_resonator(gap=0.3, radius=10, length_x=3,length_y=0,cross_section=xs,bend="bend_circular", x_span=x_span_resonator)
    spacer_ref = c << ring_beam_spacer
    spacer_ref.connect('p1', spring_beam_ref.ports['s1'],allow_width_mismatch=True)
    ring_ref.connect('p1', spacer_ref.ports['p2'],allow_width_mismatch=True)
    o1_ring = ring_ref['o1']
    o2_ring = ring_ref['o2']
    c.add_port(
        name='optical_in',
        width=o1_ring.width,
        orientation=-90,
        center=(o1_ring.center[0] - (250- x_span_resonator)/2, o1_ring.center[1] -50),
        layer=o1_ring.layer,
        port_type='optical'
    )
    c.add_port(
        name='optical_out',
        width=o2_ring.width,
        orientation=-90,
        center=(o2_ring.center[0] + (250- x_span_resonator)/2, o2_ring.center[1] -50),
        layer=o2_ring.layer,
        port_type='optical'
    )
    copied_portin = c.ports.filter(regex='optical_in')[0].copy()
    copied_portout = c.ports.filter(regex='optical_out')[0].copy()
    copied_portin.name = 'optical_in_reverse'
    copied_portout.name = 'optical_out_reverse'
    copied_portin.orientation = 90
    copied_portout.orientation = 90
    r1 = gf.routing.route_single(
        c,
        port1=o1_ring,
        port2=copied_portin,
        cross_section=xs_routing,
        )
    r2 = gf.routing.route_single(
        c,
        port1=o2_ring,
        port2=copied_portout,
        cross_section=xs_routing,
    )
    
    spring_pad = pad(size=(400, beam_length), metal_offset=10,pad_layer='PADDING')
    spring_pad_ref = c << spring_pad
    # spring_pad_ref.rotate(90)
    spring_pad_ref.movex(-beam_length-450)
    spring_pad_ref.movey(250)
    spring_pad_port_cp = spring_pad_ref.ports["E1"].copy()
    spring_pad_port_cp.center = (spring_pad_port_cp.center[0], spring_pad_port_cp.center[1]-180)
    # routing from spring beam to pad
    gf.routing.route_single(
        c,
        port1=spring_beam_ref.ports['up_anchor'],
        port2=spring_pad_port_cp,
        cross_section=xs_metal_wire,
        allow_width_mismatch=True,
        auto_taper=False
        )
    c = tmp_merge_deep_etch_mask(c)
    
    return c 
c = doubly_clamped_beam_with_ring_resonator()
c.show()

In [ ]:
c4 = gf.Component()
# extract the layers of interest into a new component
c4 << c.extract(['DEEP_ETCH', 'PADDING'])

# get the polygons dictionary and layer keys
polys = c4.get_polygons()
layer_deep = gf.get_layer(("DEEP_ETCH"))
layer_padding = gf.get_layer(("PADDING"))

# build kdb Regions safely (use get to avoid KeyError if layer missing)
reg_deep = gf.kdb.Region(polys.get(layer_deep, []))
reg_m6 = gf.kdb.Region(polys.get(layer_padding, []))

# combine regions and compute hulls
reg_deep.insert(reg_m6)
blk = gf.kdb.Region(gf.kdb.DBox(0, 0, 330e3, 1100e3))
blk.move(-160e3,-87e3)
reg_deep.insert(blk)
hulls = reg_deep.hulls().sized(-500)



# add hulls to a new component if not empty
c5 = gf.Component()
# blk = c5 << gf.components.rectangle(size=(200,1100),layer="DEEP_ETCH")
# blk.move((-68,-87))
if not hulls.is_empty():
	c5.add_polygon(hulls, layer=("DEEP_ETCH"))
c5.add_polygon(reg_deep, layer=("WG"))
bbox = gf.kdb.DPolygon(c5.bbox()).sized(50)
c5.add_polygon(bbox, layer=(7,0))
c5 << gf.boolean(c5,c5,"not",(9,0),(7,0),"DEEP_ETCH")
c5.show()

In [ ]:
def device_array():
    c = gf.Component()
    d = doubly_clamped_beam_with_ring_resonator()
    device_refs = [c << d for i in range(7)]
    for i in range(7):
        device_refs[i].movex(i*1000-1297)
    c.ports = {}
    for i, dev in enumerate(device_refs):
        c.add_ports([dev.ports["optical_in"], dev.ports["optical_out"]])
    c.auto_rename_ports()
    return c

c = gf.Component()
my_coupler_spec = partial(my_coupler,coupler=coupler_spec,waveguide_width=0.43)
gc_array = gf.components.grating_coupler_array(my_coupler_spec, pitch=250, n=16, rotation=-90, with_loopback=True, cross_section=xs_routing)
gc_array_ref = c << gc_array
gc_array_ref.move((0,-1000))
device_array_ref = c << device_array()
device_array_ref_ports = [
    gf.Port(port.name, center=port.center, width=port.width, orientation=port.orientation, layer=port.layer)
    for port in device_array_ref.ports
]


gf.routing.route_bundle(
    c,
    gc_array_ref.ports,
    device_array_ref_ports,
    
    cross_section=xs_routing,
    sort_ports=True,
    start_straight_length=10,
    separation=10
    
)
c.show()